In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# GPU SETUP - Enable GPU acceleration
#import os
#os.environ['CUDA_PATH'] = r'C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2'
#os.add_dll_directory(r'C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\bin')

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detected: {gpus}")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("⚠️ No GPU detected - running on CPU")

# Step 0: Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report

# -----------------------------
# 1) Paths and Hyperparameters
# -----------------------------
# Updated paths for local dataset
train_dir = 'datasets/Image classification-Intel datset/new_Intel_training_dataset'
test_dir  = 'datasets/Image classification-Intel datset/new_Intel_testing_dataset'
num_classes = 6
batch_size = 32
img_size = (224, 224)
epochs = 10

print(f"\n📂 Dataset paths:")
print(f"   Train: {train_dir}")
print(f"   Test:  {test_dir}")

# -----------------------------
# 2) Data Generators (no validation)
# -----------------------------
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    horizontal_flip=True,
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255,
)

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

print(f"\n✅ Data loaded:")
print(f"   Classes: {list(train_generator.class_indices.keys())}")
print(f"   Training samples: {train_generator.samples}")
print(f"   Test samples: {test_generator.samples}")

# -----------------------------
# 3) Build CNN
# -----------------------------
model = Sequential([
    tf.keras.Input(shape=(224, 224, 3)),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n🏗️ Model architecture:")
model.summary()

# -----------------------------
# 4) Train the Model (NO validation)
# -----------------------------
print("\n🚀 Starting training...")
history = model.fit(
    train_generator,
    epochs=epochs
)

# -----------------------------
# 5) Plot Training Accuracy and Loss (no validation curves)
# -----------------------------
acc = history.history['accuracy']
loss = history.history['loss']
epochs_range = range(1, len(acc) + 1)

plt.figure(figsize=(12, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.title('Training Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# -----------------------------
# 6) Final Test Evaluation
# -----------------------------
print("\n📊 Evaluating on test set...")
test_generator.reset()
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"\n✅ Test loss: {test_loss:.4f} | Test accuracy: {test_acc:.4f}")

# -----------------------------
# 7) Confusion Matrix & Report
# -----------------------------
print("\n📈 Generating predictions...")
test_generator.reset()
pred_probs = model.predict(test_generator, verbose=1)
y_pred = np.argmax(pred_probs, axis=1)
y_true = test_generator.classes
class_names = list(test_generator.class_indices.keys())

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

⚠️ No GPU detected - running on CPU

📂 Dataset paths:
   Train: datasets/Image classification-Intel datset/new_Intel_training_dataset
   Test:  datasets/Image classification-Intel datset/new_Intel_testing_dataset
Found 3000 images belonging to 6 classes.
Found 600 images belonging to 6 classes.

✅ Data loaded:
   Classes: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
   Training samples: 3000
   Test samples: 600

🏗️ Model architecture:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18

KeyboardInterrupt: 